In [14]:
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem

algorithm = NSGA2 (pop_size=500)
stop_criteria = ("n_gen", 300)

In [12]:
problem = ProblemWrapper()


NameError: name 'ProblemWrapper' is not defined

In [ ]:
result = minimize(
    problem= problem,
    algorithm=algorithm,
    termination=stop_criteria
)